In [1]:
import numpy as np
import copy
import os
import re
import random

In [2]:
def get_subdirectories(folder_path):
    subdirectories = []
    for item in os.listdir(folder_path):
        item_path = os.path.join(folder_path, item)
        if os.path.isdir(item_path):
            subdirectories.append(item)
    return subdirectories

def get_files_in_subdirectories(folder_path, file_extension='', file_contains=''):
    files = []
    for root, directories, filenames in os.walk(folder_path):
        for filename in filenames:
            if file_extension == '' and file_contains == '':
                files.append(os.path.join(root, filename))
            elif file_extension != '' and file_contains == '':
                if filename.endswith(file_extension):
                    files.append(os.path.join(root, filename))
            elif file_extension == '' and file_contains != '':
                if file_contains in filename:
                    files.append(os.path.join(root, filename))
            else:
                if file_contains in filename and filename.endswith(file_extension):
                    files.append(os.path.join(root, filename))
    return files


def extract_string(filename,format = 'stl'):
    pattern = r'\/([^/]+)\.'+format+'$'
    match = re.search(pattern, filename)
    if match:
        return match.group(1)
    else:
        return None
    
# Defining a function to convert degrees to radians.
def deg2rad(deg):
    return deg * np.pi/180

def get_rotated_pcd(pcd, x_theta, y_theta, z_theta):
    pcd_rotated = copy.deepcopy(pcd)
    R = pcd_rotated.get_rotation_matrix_from_axis_angle([x_theta, y_theta, z_theta])
    pcd_rotated.rotate(R, center=(0, 0, 0))
    return pcd_rotated

def create_folder_if_not_exists(folder_path):
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)
        print(f"Created folder: {folder_path}")
        return False
    else:
        print(f"Folder already exists: {folder_path}")
        return True

def delete_file(file_path):
    try:
        os.remove(file_path)
        # print(f"File '{file_path}' deleted successfully.")
    except FileNotFoundError:
        print(f"File '{file_path}' not found.")
    except PermissionError:
        print(f"Permission denied: unable to delete file '{file_path}'.")
    except Exception as e:
        print(f"An error occurred while deleting the file: {e}")

def create_binary_list(n, percentage_of_ones):
    if percentage_of_ones < 0 or percentage_of_ones > 100:
        raise ValueError("Percentage_of_ones must be between 0 and 100")

    num_ones = int(n * (percentage_of_ones / 100))
    num_zeros = n - num_ones

    binary_list = [1] * num_ones + [0] * num_zeros
    random.shuffle(binary_list)

    return binary_list


In [3]:
cwd = os.getcwd()
parent_dir = os.path.dirname(cwd)
data_path = parent_dir+'/data/'
cad_path = data_path + 'cad-models-threads/'
print(cad_path)
file_extension = '.stl'
path_save = data_path+'train_threads/imagesO/'
print(path_save)
files = get_files_in_subdirectories(cad_path,file_extension)

/Users/rodolfocacacho/Documents/Documents/MAI/Project module/3d_mai/data/cad-models-threads/
/Users/rodolfocacacho/Documents/Documents/MAI/Project module/3d_mai/data/train_threads/imagesO/


In [16]:
cwd = os.getcwd()
parent_dir = os.path.dirname(cwd)
data_path = parent_dir+'/data/'
cad_path = data_path + 'cad-models-threads/'
print(cad_path)
file_extension = '.stl'
path_imgs = data_path+'train_threads_b/imagesO/'
path_save = path_imgs.replace('imagesO','images')
path_val_save = path_save.replace('train','val')
labels_save = data_path+'train_threads_b/labels/'
labels_val_save = data_path+'val_threads_b/labels/'
path_save_bb = data_path+'train_threads_b/images_wbb/'
print(path_save)
files = get_files_in_subdirectories(cad_path,file_extension)
val_save = ''

exists_save = create_folder_if_not_exists(path_save)
if exists_save:
    images_del = get_files_in_subdirectories(path_save,'png')
    for imdel in images_del:
        delete_file(imdel)
exists_lb = create_folder_if_not_exists(labels_save)
if exists_lb:
    images_del = get_files_in_subdirectories(labels_save,'txt')
    for imdel in images_del:
        delete_file(imdel)
exists_wbb = create_folder_if_not_exists(path_save_bb)
if exists_wbb:
    images_del = get_files_in_subdirectories(path_save_bb,'png')
    for imdel in images_del:
        delete_file(imdel)
exists_ival = create_folder_if_not_exists(path_val_save)
if exists_ival:
    images_del = get_files_in_subdirectories(path_val_save,'png')
    for imdel in images_del:
        delete_file(imdel)
exists_l_val = create_folder_if_not_exists(labels_val_save)
if exists_l_val:
    images_del = get_files_in_subdirectories(labels_val_save,'txt')
    for imdel in images_del:
        delete_file(imdel)

/Users/rodolfocacacho/Documents/Documents/MAI/Project module/3d_mai/data/cad-models-threads/
/Users/rodolfocacacho/Documents/Documents/MAI/Project module/3d_mai/data/train_threads_b/images/
Folder already exists: /Users/rodolfocacacho/Documents/Documents/MAI/Project module/3d_mai/data/train_threads_b/images/
Folder already exists: /Users/rodolfocacacho/Documents/Documents/MAI/Project module/3d_mai/data/train_threads_b/labels/
Folder already exists: /Users/rodolfocacacho/Documents/Documents/MAI/Project module/3d_mai/data/train_threads_b/images_wbb/
Folder already exists: /Users/rodolfocacacho/Documents/Documents/MAI/Project module/3d_mai/data/val_threads_b/images/
Folder already exists: /Users/rodolfocacacho/Documents/Documents/MAI/Project module/3d_mai/data/val_threads_b/labels/


In [15]:
import cv2
images_move = get_files_in_subdirectories(path_imgs,'.png')
test_list = create_binary_list(len(images_move), 20)

unique_classes = []

for index, jpg in enumerate(images_move):
    # Load the image
    image = cv2.imread(jpg)

    # Convert the image to grayscale
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # Apply Gaussian blur to reduce noise
    blurred = cv2.GaussianBlur(gray, (5, 5), 0)

    # Perform edge detection
    edges = cv2.Canny(blurred, 50, 150)

    # Find contours in the edge-detected image
    contours, _ = cv2.findContours(edges, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Find the convex hull of the contours
    convex_hull = cv2.convexHull(np.vstack(contours))

    # Create a copy of the image for drawing the bounding box
    result_image = image.copy()

    # Calculate the bounding box of the convex hull
    x, y, w, h = cv2.boundingRect(convex_hull)

    x1 = x
    y1 = y
    x2 = x + w
    y2 = y + h
    w_reduce = 960
    h_reduce = 500
    # Calculate the maximum x and y coordinates for the new location
    max_x = image.shape[1] - w - int(w_reduce/2)
    max_y = image.shape[0] - h - int(h_reduce/2)

    # Generate random x and y coordinates for the new location
    new_x = np.random.randint(int(w_reduce/2), max_x + 1)
    new_y = np.random.randint(int(h_reduce/2), max_y + 1)

    # Extract the object
    object_region = image[y1:y2, x1:x2].copy()

    # Paste the object into the new random location
    image[y1:y2, x1:x2] = [255, 255, 255]
    image[new_y:new_y+h, new_x:new_x+w] = object_region

    # Save the edited image
    if test_list[index] == 0:
        jpg_saving_path = jpg.replace("imagesO", "images")
        cv2.imwrite(jpg_saving_path, image)
    else:
        jpg_saving_path = jpg.replace("imagesO", "images")
        jpg_saving_path = jpg_saving_path.replace("train", "val")
        # print(jpg_saving_path)
        cv2.imwrite(jpg_saving_path, image)
    
    # Create a copy of the image for drawing the bounding box
    result_image = image.copy()

    # Draw the bounding box on the result image
    cv2.rectangle(result_image, (new_x, new_y), (new_x + w, new_y + h), (0, 255, 0), 2)
    bounding_box_saving_path = jpg.replace("imagesO", "images_wbb")
    cv2.imwrite(bounding_box_saving_path, result_image)

    center_x = new_x + (w) / 2
    center_y = new_y + (h) / 2
    relative_width = (w) / image.shape[1]
    #print(relative_width)
    relative_height = (h) / image.shape[0]
    #print(relative_height)
    relative_center_x= center_x / image.shape[1]
    relative_center_y= center_y / image.shape[0]

    # Open a file for writing the results
    filename = jpg.split('/')[-1].split('.')[0]
    classname = filename.split('_x')[0]
    if classname not in unique_classes:
        unique_classes.append(classname)
    encoded_class = unique_classes.index(classname)
    if test_list[index] == 0:
        with open(labels_save + filename + '.txt', 'w') as file:
            file.write(f"{encoded_class} {relative_center_x:.5f} {relative_center_y:.5f} {relative_width:.5f} {relative_height:.5f}")
    else:
        with open(labels_save.replace('train','val') + filename + '.txt', 'w') as file:
            file.write(f"{encoded_class} {relative_center_x:.5f} {relative_center_y:.5f} {relative_width:.5f} {relative_height:.5f}")
    

/Users/rodolfocacacho/Documents/Documents/MAI/Project module/3d_mai/data/val_threads_b/images/m4_10mm_x-1_y-1_z-3_z-z3oom.png
/Users/rodolfocacacho/Documents/Documents/MAI/Project module/3d_mai/data/val_threads_b/images/din125_m4_x-4_y-1_z-0_z-z1oom.png
/Users/rodolfocacacho/Documents/Documents/MAI/Project module/3d_mai/data/val_threads_b/images/m4_20mm_x-4_y-4_z-0_z-z3oom.png
/Users/rodolfocacacho/Documents/Documents/MAI/Project module/3d_mai/data/val_threads_b/images/m4_20mm_x-1_y-2_z-3_z-z3oom.png
/Users/rodolfocacacho/Documents/Documents/MAI/Project module/3d_mai/data/val_threads_b/images/dst4_x-1_y-3_z-3_z-z1oom.png
/Users/rodolfocacacho/Documents/Documents/MAI/Project module/3d_mai/data/val_threads_b/images/dst4_x-3_y-3_z-2_z-z2oom.png
/Users/rodolfocacacho/Documents/Documents/MAI/Project module/3d_mai/data/val_threads_b/images/m4_10mm_x-2_y-0_z-0_z-z3oom.png
/Users/rodolfocacacho/Documents/Documents/MAI/Project module/3d_mai/data/val_threads_b/images/asy_m4_nut_screw_10mm_x-3_y-